In [ ]:
!pip install copernicusmarine

In [1]:
from pprint import pprint
import xarray as xr
import copernicusmarine as copernicusmarine
print(copernicusmarine.__version__)
#only for the first time: 
#copernicusmarine.login() 

1.1.0


In [58]:
catalogue = copernicusmarine.describe(
    include_datasets=True,
#    contains = ['European North West Shelf - Ocean Physics '],
#     contains = ['Atlantic-Iberian Biscay Irish- Ocean Physics']   
#    contains = ['thetao']
#
    contains=['Ocean Physics']
)
for index, product in enumerate(catalogue['products']):
    print(index, product['title'], product['product_id'])
    
    for index2, dataset in enumerate(product['datasets']):
       # if 'dataset_id' in dataset and 'H' in dataset['dataset_id']:        
        print(index,index2, 'dataset_id',dataset['dataset_id'], dataset['dataset_name'])

Fetching catalog: 100%|██████████| 3/3 [00:10<00:00,  3.52s/it]


0 Arctic Ocean Physics Analysis and Forecast ARCTIC_ANALYSISFORECAST_PHY_002_001
0 0 dataset_id cmems_mod_arc_phy_anfc_6km_detided_P1D-m Arctic Ocean Physics Analysis and Forecast, 6.25 km daily mean
0 1 dataset_id cmems_mod_arc_phy_anfc_6km_detided_P1M-m Arctic Ocean Physics Analysis, 6.25km monthly mean
1 Arctic Ocean Physics Reanalysis ARCTIC_MULTIYEAR_PHY_002_003
1 0 dataset_id cmems_mod_arc_phy_my_topaz4_P1D-m Arctic Ocean Physics Reanalysis
1 1 dataset_id cmems_mod_arc_phy_my_topaz4_P1M Arctic Ocean Physics Reanalysis
1 2 dataset_id cmems_mod_arc_phy_my_topaz4_P1Y Arctic Ocean Physics Reanalysis
2 Global Ocean Physics Analysis and Forecast GLOBAL_ANALYSISFORECAST_PHY_001_024
2 0 dataset_id cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m daily mean fields from Global Ocean Physics Analysis and Forecast updated Daily
2 1 dataset_id cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m daily mean fields from Global Ocean Physics Analysis and Forecast updated Daily
2 2 dataset_id cmems_mod_glo_phy-thet

In [13]:
catalogue['products'][0]['title']

'Atlantic-Iberian Biscay Irish- Ocean Physics Analysis and Forecast'

In [15]:
name='cmems_mod_ibi_phy_anfc_0.027deg'
name='cmems_mod_nws_phy_anfc_0.027deg'

catalogue = copernicusmarine.describe(
    include_datasets=True,
    contains = [name]
)

Fetching catalog: 100%|██████████| 3/3 [00:11<00:00,  3.72s/it]


In [53]:
uris_by_key = {}  # Dictionary to store URIs by key
def get_copernicus_data(name,format='arco-geo-series' ):
    #format='arco-geo-series' or 'arco-time-series'
    catalogue = copernicusmarine.describe(
    include_datasets=True,
    contains = [name],
    )
    for value in catalogue['products'][0]['datasets']:
        dataset_id = value['dataset_id']
        if any(substring in dataset_id for substring in ['static', '2D_PT1H-m', '3D_PT1H-m']):
            uris = []
            for service in value['versions'][0]['parts'][0]['services']:
                service_name = service['service_type']['service_name']
                if service_name in [format, 'static-arco']:
                    uris.append(service['uri'])
            uris_by_key[dataset_id] = uris
#    bbox = {"latitude": [48, 49], "longitude": [-6, -4]} 
    #.sel(latitude=slice(*bbox["latitude"]), longitude=slice(*bbox["longitude"]))
    thetao=(
        xr.open_dataset(uris_by_key[name+'-3D_PT1H-m'][0],
        engine='zarr',
        chunks={})[['thetao']]
    #.sel(latitude=slice(*bbox["latitude"]), longitude=slice(*bbox["longitude"]))
    )
    zos=(
        xr.open_dataset(uris_by_key[name+'-2D_PT1H-m'][0],
        engine='zarr',
        chunks={}).zos
         #.sel(latitude=slice(*bbox["latitude"]), longitude=slice(*bbox["longitude"]))
        )
    deptho=(
        xr.open_dataset(uris_by_key[name+'-3D_static'][0],
        engine='zarr',
        chunks={}).deptho
        #.sel(latitude=slice(*bbox["latitude"]), longitude=slice(*bbox["longitude"]))
        )
    mask=deptho.isnull()
    print('zos min,max value',zos.shape,zos.latitude[0:2].data,zos.latitude[-3:-1].data)
    print('deptho min,max value',deptho.shape,deptho.latitude[0:2].data,deptho.latitude[-3:-1].data)
    print('thetao min,max value',thetao.thetao.shape,thetao.latitude[0:2].data,thetao.latitude[-3:-1].data)
    ds=(
        thetao.rename({"thetao": "TEMP"})
        .assign(
                {
                    "XE": zos,
                    "H0": (deptho
                    ),
                    "mask": (mask
                    ),
                }
        )).rename({"latitude": "lat", "longitude": "lon", "elevation": "depth"})
    return ds
    

## name='cmems_mod_ibi_phy_anfc_0.027deg'
check if not mix with u,v,t coordinate on zarr?
compare with netcdf original file lat lon coordinate? 
only available from 2023-03-30, which does not corresopnds to the description.
check for both zarr format the time availavility 

In [54]:
get_copernicus_data(name='cmems_mod_ibi_phy_anfc_0.027deg' )

Fetching catalog: 100%|██████████| 3/3 [00:10<00:00,  3.59s/it]


zos min,max value (26472, 1081, 865) [26.       26.027779] [55.944443 55.97222 ]
deptho min,max value (1081, 865) [26.       26.027779] [55.945362 55.97314 ]
thetao min,max value (9048, 49, 1014, 840) [27.47227 27.50005] [55.55646 55.58424]


<xarray.Dataset>
Dimensions:  (depth: 49, lat: 1014, lon: 840, time: 9048)
Coordinates:
  * depth    (depth) float32 -5.275e+03 -4.833e+03 -4.405e+03 ... -1.541 -0.494
  * lat      (lat) float32 27.47 27.5 27.53 27.56 ... 55.53 55.56 55.58 55.61
  * lon      (lon) float32 -18.56 -18.53 -18.5 -18.47 ... 4.695 4.723 4.751
  * time     (time) datetime64[ns] 2023-03-30 ... 2024-04-09T23:00:00
Data variables:
    TEMP     (time, depth, lat, lon) float32 dask.array<chunksize=(1, 1, 1014, 840), meta=np.ndarray>
    XE       (time, lat, lon) float32 dask.array<chunksize=(1, 1014, 840), meta=np.ndarray>
    H0       (lat, lon) float32 dask.array<chunksize=(1014, 840), meta=np.ndarray>
    mask     (lat, lon) float64 dask.array<chunksize=(1014, 840), meta=np.ndarray>
Attributes: (12/21)
    Conventions:     CF-1.0
    bulletin_date:   2023-03-31
    bulletin_type:   Operational
    contact:         mailto: servicedesk.cmems@mercator-ocean.eu
    domain_name:     IBI36
    easting:         longitude
    ...              ...
    northing:        latitude
    references:      http://marine.copernicus.eu
    source:          NEMO3.6
    title:           Ocean 3D IBI hourly mean fields
    z_max:           5274.7841796875
    z_min:           0.49399998784065247

## name='cmems_mod_nws_phy_anfc_0.027deg'
check if not mix with u,v,t coordinate on zarr?
compare with netcdf original file lat lon coordinate? 
only available from 2023-07-02, which does not corresopnds to the description.
check for both zarr format the time availavility 
verify if the data resolution is exactry same as what we have on datarmor


In [55]:
get_copernicus_data(name='cmems_mod_nws_phy_anfc_0.027deg' )


Fetching catalog: 100%|██████████| 3/3 [00:10<00:00,  3.54s/it]


zos min,max value (22848, 551, 936) [46.003643 46.03142 ] [61.22633 61.25411]
deptho min,max value (551, 936) [46.00364  46.031418] [61.22633 61.25411]
thetao min,max value (6792, 50, 551, 936) [46.003643 46.03142 ] [61.22633 61.25411]


<xarray.Dataset>
Dimensions:  (depth: 50, lat: 551, lon: 936, time: 6792)
Coordinates:
  * depth    (depth) float32 -5.728e+03 -5.275e+03 -4.833e+03 ... -1.541 -0.494
  * lat      (lat) float32 46.0 46.03 46.06 46.09 ... 61.2 61.23 61.25 61.28
  * lon      (lon) float32 -16.0 -15.97 -15.94 -15.91 ... 9.921 9.949 9.977
  * time     (time) datetime64[ns] 2023-07-02 ... 2024-04-09T23:00:00
Data variables:
    TEMP     (time, depth, lat, lon) float32 dask.array<chunksize=(2, 1, 551, 936), meta=np.ndarray>
    XE       (time, lat, lon) float32 dask.array<chunksize=(2, 551, 936), meta=np.ndarray>
    H0       (lat, lon) float32 dask.array<chunksize=(551, 936), meta=np.ndarray>
    mask     (lat, lon) float64 dask.array<chunksize=(551, 936), meta=np.ndarray>
Attributes: (12/13)
    Conventions:     CF-1.8
    comment:         
    contact:         https://marine.copernicus.eu/contact
    domain_name:     NWS36
    field_date:      20231027
    field_type:      mean
    ...              ...
    forecast_type:   forecast
    institution:     Nologin (Spain)
    licence:         https://marine.copernicus.eu/user-corner/service-commitm...
    references:      http://marine.copernicus.eu/
    source:          NEMO3.6
    title:           Ocean 3D hourly mean fields for the North West Shelf (NW...